# The 2018 Final Four games are played under a full moon. When was the last time there was an upset in a Final Four game under a full moon?

## Step 1

### Let's start by taking a look at a dataset of the phases of the moon. The moon has [4 principal phases](https://en.wikipedia.org/wiki/Lunar_phase#Phases_of_the_Moon): the new moon, the first quarter, the full moon, and the last quarter. A full moon occurs when Earth is located directly between the sun and the moon, causing the part of the moon closest to Earth to appear fully sunlit, while the far side is completely dark.

In [0]:
from pandas.io import gbq
project_id = '[YOUR_PROJECT_ID]'

In [14]:
moons_q = """
SELECT phase, phase_emoji, peak_datetime
FROM `bigquery-public-data.moon_phases.moon_phases`
ORDER BY peak_datetime ASC
"""

moons = gbq.read_gbq(query=moons_q, dialect ='standard', project_id=project_id)
moons.head(25)

Requesting query... ok.
Job ID: job_NMtV3iD9oiRzJ_XQaRQJ-_keIg5O
Query running...
Query done.
Cache hit.

Retrieving results...
Got 19789 rows.

Total time taken 2.56 s.
Finished at 2018-03-05 07:33:15.


,phase,phase_emoji,peak_datetime
0,Full Moon,🌕,1700-01-05T10:30:00
1,Last Quarter,🌗,1700-01-12T03:34:00
2,New Moon,🌑,1700-01-20T04:20:00
3,First Quarter,🌓,1700-01-28T05:13:00
4,Full Moon,🌕,1700-02-03T21:05:00
5,Last Quarter,🌗,1700-02-10T18:59:00
6,New Moon,🌑,1700-02-18T23:33:00
7,First Quarter,🌓,1700-02-26T16:37:00
8,Full Moon,🌕,1700-03-05T07:38:00
9,Last Quarter,🌗,1700-03-12T12:29:00


## Step 2

### These timestamps are in [UTC time](https://en.wikipedia.org/wiki/Coordinated_Universal_Time), so we'll need to convert these to a different timezone if we want to find the date of the full moon in the United States:

In [15]:
moons_et_q = """
SELECT 
  phase,
  phase_emoji,
  DATETIME(TIMESTAMP(peak_datetime), "America/New_York") phase_peak_nyc
FROM `bigquery-public-data.moon_phases.moon_phases`
ORDER BY peak_datetime ASC
"""

moons_et = gbq.read_gbq(query=moons_et_q, dialect ='standard', project_id=project_id)
moons_et.head(25)

Requesting query... ok.
Job ID: job_7UlI_-67yriq6C4OZcXxIplbRik4
Query running...
Query done.
Cache hit.

Retrieving results...
Got 19789 rows.

Total time taken 2.13 s.
Finished at 2018-03-05 07:33:23.


,phase,phase_emoji,phase_peak_nyc
0,Full Moon,🌕,1700-01-05T05:33:58
1,Last Quarter,🌗,1700-01-11T22:37:58
2,New Moon,🌑,1700-01-19T23:23:58
3,First Quarter,🌓,1700-01-28T00:16:58
4,Full Moon,🌕,1700-02-03T16:08:58
5,Last Quarter,🌗,1700-02-10T14:02:58
6,New Moon,🌑,1700-02-18T18:36:58
7,First Quarter,🌓,1700-02-26T11:40:58
8,Full Moon,🌕,1700-03-05T02:41:58
9,Last Quarter,🌗,1700-03-12T07:32:58


## Step 3

### Since the exact moment of the full moon peak may or may not occur “at night” in whatever time zone you happen to be (or, for our purposes, wherever basketball games are being played), we need to come up with a reasonable window of time around the full moon peak in which a game might have transpired. 

### NCAA Final Four games tend to happen between 6 PM and 12 PM EST, so let's start with that as our baseline game window. We'll then add an 18-hour cushion on either side of the game time to account for the passage of any moon phase peak. This establishes a 42-hour window to capture tournament games near or during peak phases of the moon, and excludes games that occurred outside of those 42-hour windows.

### Now that our window is defined, we'll use an *inner* join so that we only see games that match with our moon phase windows. Here are the games that were played within 18 hours of any moon phase peak:

In [13]:
games_all_phases_q = """
SELECT
  game_date,
  round, 
  moon.phase,
  moon.phase_emoji,
  DATETIME(TIMESTAMP(peak_datetime), "America/New_York") phase_peak_nyc,
  win_seed, 
  win_market, 
  lose_seed, 
  lose_market
FROM `bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games` as game
INNER JOIN `bigquery-public-data.moon_phases.moon_phases` as moon on 
  (DATETIME(TIMESTAMP(peak_datetime), "America/New_York") > DATETIME(game_date) AND  # <<<<< this line is the lower limit, restricting the results to just games where the phase peak occurs within 18 hours of the game start time.
  DATETIME(TIMESTAMP(peak_datetime), "America/New_York") < DATETIME_ADD(DATETIME(game_date), INTERVAL 42 HOUR))# <<<<< this line is the upper limit, restricting the results to just games where the phase peak occurs within 18 hours of the game end time.
ORDER BY game_date DESC
"""

games_all_phases = gbq.read_gbq(query=games_all_phases_q, dialect ='standard', project_id=project_id)
games_all_phases.head(25)

Requesting query... ok.
Job ID: job_r-e7ZqqeFC2YlIgn34yo00-0T0wC
Query running...
Query done.
Cache hit.

Retrieving results...
Got 435 rows.

Total time taken 1.05 s.
Finished at 2018-03-05 07:32:50.


,game_date,round,phase,phase_emoji,phase_peak_nyc,win_seed,win_market,lose_seed,lose_market
0,2017-04-03,2,First Quarter,🌓,2017-04-03T14:39:00,01,North Carolina,01,Gonzaga
1,2017-03-19,32,Last Quarter,🌗,2017-03-20T11:58:00,07,South Carolina,02,Duke
2,2017-03-19,32,Last Quarter,🌗,2017-03-20T11:58:00,01,Kansas,09,Michigan State
3,2017-03-19,32,Last Quarter,🌗,2017-03-20T11:58:00,01,North Carolina,08,Arkansas
4,2017-03-19,32,Last Quarter,🌗,2017-03-20T11:58:00,03,Oregon,11,Rhode Island
5,2017-03-19,32,Last Quarter,🌗,2017-03-20T11:58:00,03,Baylor,11,USC
6,2017-03-19,32,Last Quarter,🌗,2017-03-20T11:58:00,02,Kentucky,10,Wichita State
7,2017-03-19,32,Last Quarter,🌗,2017-03-20T11:58:00,07,Michigan,02,Louisville
8,2017-03-19,32,Last Quarter,🌗,2017-03-20T11:58:00,03,UCLA,06,Cincinnati
9,2016-03-15,68,First Quarter,🌓,2016-03-15T13:03:00,16,Florida Gulf Coast,16,Fairleigh Dickinson


## Step 4

### Now, let's limit it to only games on the full moon.

In [16]:
games_full_moons_q = """
SELECT
  game_date,
  round, 
  moon.phase,
  moon.phase_emoji,
  DATETIME(TIMESTAMP(peak_datetime), "America/New_York") phase_peak_nyc,
  win_seed, 
  win_market, 
  lose_seed, 
  lose_market
FROM `bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games` as game
INNER JOIN `bigquery-public-data.moon_phases.moon_phases` as moon on 
  (DATETIME(TIMESTAMP(peak_datetime), "America/New_York") > DATETIME(game_date) AND  # <<<<< this line is the lower limit, restricting the results to just games where the phase peak occurs within 18 hours of the game start time.
  DATETIME(TIMESTAMP(peak_datetime), "America/New_York") < DATETIME_ADD(DATETIME(game_date), INTERVAL 42 HOUR))# <<<<< this line is the upper limit, restricting the results to just games where the phase peak occurs within 18 hours of the game end time.
WHERE moon.phase_emoji = "🌕" # <<<<<<<<<<<< this is the new line
ORDER BY game_date DESC
"""

games_full_moons = gbq.read_gbq(query=games_full_moons_q, dialect ='standard', project_id=project_id)
games_full_moons.head(25)

Requesting query... ok.
Job ID: job_NhYoUXABaoNKzb-23OrHS60exWp7
Query running...
Query done.
Processed: 608.7 KB
Standard price: $0.00 USD

Retrieving results...
Got 137 rows.

Total time taken 2.45 s.
Finished at 2018-03-05 07:36:54.


,game_date,round,phase,phase_emoji,phase_peak_nyc,win_seed,win_market,lose_seed,lose_market
0,2015-04-04,4,Full Moon,🌕,2015-04-04T08:05:00,01,Duke,07,Michigan State
1,2015-04-04,4,Full Moon,🌕,2015-04-04T08:05:00,01,Wisconsin,01,Kentucky
2,2011-03-19,32,Full Moon,🌕,2011-03-19T14:10:00,03,Connecticut,06,Cincinnati
3,2011-03-19,32,Full Moon,🌕,2011-03-19T14:10:00,03,BYU,11,Gonzaga
4,2011-03-19,32,Full Moon,🌕,2011-03-19T14:10:00,04,Wisconsin,05,Kansas State
5,2011-03-19,32,Full Moon,🌕,2011-03-19T14:10:00,04,Kentucky,05,West Virginia
6,2011-03-19,32,Full Moon,🌕,2011-03-19T14:10:00,08,Butler,01,Pittsburgh
7,2011-03-19,32,Full Moon,🌕,2011-03-19T14:10:00,12,Richmond,13,Morehead State
8,2011-03-19,32,Full Moon,🌕,2011-03-19T14:10:00,02,Florida,07,UCLA
9,2011-03-19,32,Full Moon,🌕,2011-03-19T14:10:00,02,San Diego State,07,Temple


### Ok, now we're getting somewhere. Of the 2117 games played in the NCAA tournament since 1985, only 137 have been played under a full moon.

### 4A: As an aside, that's only slightly more than you'd expect. According to Wikipedia, a [Synodic lunar month](https://en.wikipedia.org/wiki/Lunar_month#Synodic_month) averages 29.530587981 days. We'd expect this many games under our full moon window:

In [17]:
(42.0/24) * (2117/29.530587981)


125.45466424114679

## Step 5

### Back to business... as we know, the Final Four in 2018 is on a full moon (the peak is at 8:37AM ET the morning of the games), so let's only look at Final Four games.

In [18]:
games_full_moons_4_q = """
SELECT
  game_date,
  round, 
  moon.phase,
  moon.phase_emoji,
  DATETIME(TIMESTAMP(peak_datetime), "America/New_York") phase_peak_nyc,
  win_seed, 
  win_market, 
  lose_seed, 
  lose_market
FROM `bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games` as game
INNER JOIN `bigquery-public-data.moon_phases.moon_phases` as moon on 
  (DATETIME(TIMESTAMP(peak_datetime), "America/New_York") > DATETIME(game_date) AND  # <<<<< this line is the lower limit, restricting the results to just games where the phase peak occurs within 18 hours of the game start time.
  DATETIME(TIMESTAMP(peak_datetime), "America/New_York") < DATETIME_ADD(DATETIME(game_date), INTERVAL 42 HOUR))# <<<<< this line is the upper limit, restricting the results to just games where the phase peak occurs within 18 hours of the game end time.
WHERE moon.phase_emoji = "🌕"
AND round = 4 # <<<<<<<<<<<< this is the new line
ORDER BY game_date DESC
"""

games_full_moons_4 = gbq.read_gbq(query=games_full_moons_4_q, dialect ='standard', project_id=project_id)
games_full_moons_4.head()


Requesting query... ok.
Job ID: job_8PFaF07_ALxKE7DkLSZLZ522nHrC
Query running...
Query done.
Processed: 608.7 KB
Standard price: $0.00 USD

Retrieving results...
Got 6 rows.

Total time taken 2.39 s.
Finished at 2018-03-05 07:39:34.


,game_date,round,phase,phase_emoji,phase_peak_nyc,win_seed,win_market,lose_seed,lose_market
0,2015-04-04,4,Full Moon,🌕,2015-04-04T08:05:00,01,Wisconsin,01,Kentucky
1,2015-04-04,4,Full Moon,🌕,2015-04-04T08:05:00,01,Duke,07,Michigan State
2,1991-03-30,4,Full Moon,🌕,1991-03-30T02:17:00,02,Duke,01,UNLV
3,1991-03-30,4,Full Moon,🌕,1991-03-30T02:17:00,03,Kansas,01,North Carolina
4,1988-04-02,4,Full Moon,🌕,1988-04-02T04:21:00,01,Oklahoma,01,Arizona


## Step 6

### It's only 6 games! It's easy to see, but let's run the query to only return the upsets. Since we've organized the tournament_games dataset to show the seed of the winning team and the seed of the losing team, it's easy to find them: they're the games where the winning seed is a larger number than the losing seed.

In [19]:
upsets_full_moons_4_q = """
SELECT
  game_date,
  round, 
  moon.phase,
  moon.phase_emoji,
  DATETIME(TIMESTAMP(peak_datetime), "America/New_York") phase_peak_nyc,
  win_seed, 
  win_market, 
  lose_seed, 
  lose_market
FROM `bigquery-public-data.ncaa_basketball.mbb_historical_tournament_games` as game
INNER JOIN `bigquery-public-data.moon_phases.moon_phases` as moon on 
  (DATETIME(TIMESTAMP(peak_datetime), "America/New_York") > DATETIME(game_date) AND  # <<<<< this line is the lower limit, restricting the results to just games where the phase peak occurs within 18 hours of the game start time.
  DATETIME(TIMESTAMP(peak_datetime), "America/New_York") < DATETIME_ADD(DATETIME(game_date), INTERVAL 42 HOUR))# <<<<< this line is the upper limit, restricting the results to just games where the phase peak occurs within 18 hours of the game end time.
WHERE moon.phase_emoji = "🌕"
AND round = 4
AND win_seed > lose_seed # <<<<<<<<<<<< this is the new line
ORDER BY game_date DESC
"""

upsets_full_moons_4 = gbq.read_gbq(query=upsets_full_moons_4_q, dialect ='standard', project_id=project_id)
upsets_full_moons_4.head()



Requesting query... ok.
Job ID: job_F9i3fuE7O9xREHRho4rdNM0IuPat
Query running...
Query done.
Processed: 608.7 KB
Standard price: $0.00 USD

Retrieving results...
Got 3 rows.

Total time taken 1.48 s.
Finished at 2018-03-05 07:40:40.


,game_date,round,phase,phase_emoji,phase_peak_nyc,win_seed,win_market,lose_seed,lose_market
0,1991-03-30,4,Full Moon,🌕,1991-03-30T02:17:00,03,Kansas,01,North Carolina
1,1991-03-30,4,Full Moon,🌕,1991-03-30T02:17:00,02,Duke,01,UNLV
2,1988-04-02,4,Full Moon,🌕,1988-04-02T04:21:00,06,Kansas,02,Duke


# There have been three upsets in the Final Four under a full moon since 1985, two of which both happened in 1991!